## Build a News ETL Data Pipeline Using Python and SQLite

### Task 1: Import Libraries and Connect to the News API

In [1]:
import pandas as pd
import sqlite3
import logging
from newsapi import NewsApiClient

In [2]:
news_api_key = "<YOUR_NEWS_API_KEY>" 
news_api = NewsApiClient(api_key=news_api_key)

### Task 2: Retrieve and Print News Articles

In [9]:
def extract_news_data():
    try:
        result = news_api.get_everything(q="AI", language="en",sort_by='publishedAt')
        logging.info("Connection is successful.")
        return result["articles"]
    except:
        logging.error("Connection is unsuccessful.")
        return None

articles = extract_news_data()

print(articles[:3])

[{'source': {'id': None, 'name': 'C21media.net'}, 'author': 'Jonathan Webdale', 'title': 'AI content licensing outfit Calliope Networks snapped up by New York newbie Protege', 'description': 'Calliope Networks, the Los Angeles-based start-up positioning itself as an intermediary between content owners and AI companies willing to pay millions for licensed film and TV, has been snapped up by New York-headquartered Protege.', 'url': 'https://www.c21media.net/news/ai-content-licensing-outfit-calliope-networks-snapped-up-by-new-york-newbie-protege/', 'urlToImage': 'https://cdn.c21media.net/wp-content/uploads/2024/12/davis-and-samuals.jpg', 'publishedAt': '2024-12-18T16:30:16Z', 'content': 'This premium content is only accessible with the correct C21 subscription.If you already have a subscription, click here to sign in.\r\nIf you do not have a current subscription, click here to view our… [+104 chars]'}, {'source': {'id': None, 'name': 'Gizmodo.com'}, 'author': 'Joe Tilleli', 'title': 'This

### Task 3: Clean Author Column

In [10]:
def clean_author_column(text):
    try:
        return text.split(",")[0].title()
    except AttributeError:
        return "No Author"

### Task 4: Transform News Data

In [11]:
def transform_news_data(articles):
    article_list = []
    for i in articles:
        article_list.append([value.get("name", 0) if key == "source" else value for key, value in i.items() if key in ["author", "title", "publishedAt", "content", "url", "source"]])

    df = pd.DataFrame(article_list, columns=["Source", "Author Name", "News Title", "URL", "Date Published", "Content"])

    df["Date Published"] = pd.to_datetime(df["Date Published"]).dt.strftime('%Y-%m-%d %H:%M:%S')
    df["Author Name"] = df["Author Name"].apply(clean_author_column)
 
    return df

transformed_data = transform_news_data(articles)

print(transformed_data)

           Source           Author Name  \
0    C21media.net      Jonathan Webdale   
1     Gizmodo.com           Joe Tilleli   
2         ANSA.it             No Author   
3      TechCrunch           Sarah Perez   
4     TalkAndroid         Ayomide Sadiq   
..            ...                   ...   
95  GlobeNewswire           Thoughtspot   
96       Phoronix       Michael Larabel   
97    VentureBeat        Dean Takahashi   
98  GlobeNewswire  Research And Markets   
99      TechRadar            Craig Hale   

                                           News Title  \
0   AI content licensing outfit Calliope Networks ...   
1   This AI-Powered Oral-B Electric Toothbrush Is ...   
2   Trovati e raccolti diversi corpi a Damasco, Siria   
3   Flipboard lauches Surf, a new app for browing ...   
4   S25 Ultra Will Have The Slimmest Bezels Yet; C...   
..                                                ...   
95  ThoughtSpot Appoints Francois Lopitaux to Lead...   
96  MemryX MX3 M.2 Module D

### Task 5: Load the Data into SQLite Database


In [12]:
def load_news_data(data):
    with sqlite3.connect("/usercode/news_data.sqlite") as connection:
        cursor = connection.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS news_table (
                "Source" VARCHAR(30),
                "Author Name" TEXT,
                "News Title" TEXT,
                "URL" TEXT,
                "Date Published" TEXT,
                "Content" TEXT
            )
        ''')
    data.to_sql(name="news_table", con=connection, index=False, if_exists="append")
 
load_news_data(transformed_data)

### Task 8: Verify Data Loading

In [3]:
with sqlite3.connect("/usercode/etl_news_data.sqlite") as connection:
    df = pd.read_sql("SELECT * FROM news_table;", connection)
df.head()

,Source,Author Name,News Title,URL,Date Published,Content
0,Wired,Will Knight,"Botto, the Millionaire AI Artist, Is Getting a...",https://www.wired.com/story/botto-the-milliona...,2024-12-18 17:00:00,"Its an interesting idea, and it is fun to see ..."
1,[Removed],No Author,[Removed],https://removed.com,2024-12-18 13:30:29,[Removed]
2,Wired,Molly Higgins,Petkit Purobot Ultra Review: Taking Litter Box...,https://www.wired.com/review/petkit-purobot-ul...,2024-12-18 14:04:00,Screenshots courtesy of Molly Higgins\r\nSeein...
3,Gizmodo.com,Mike Fazioli,Samsung Galaxy Watch 7 Is Yours for Under $100...,https://gizmodo.com/samsung-galaxy-watch-7-is-...,2024-12-18 15:30:38,If you’re one of the many people nervously eye...
4,The Verge,Chris Welch,LG will bring its wireless TV tech to Mini LED...,https://www.theverge.com/2024/12/18/24324162/l...,2024-12-18 14:31:22,LG will bring its wireless TV tech to Mini LED...
